In [1]:
import requests
import time
import pandas as pd

In [2]:
base_url = "http://localhost:8000" 

In [3]:
def post_tweet_for_prediction(tweet, user):
    response = requests.post(f'{base_url}/predict', json={"text": tweet, "user": user})
    return response.json()

## user posting tweet

In [4]:
def store_posted_tweet(tweet_id, retweet_id, user_id, text, likes, retweets, safety_status):
    data = {
        "tweet_id": tweet_id,
        "retweet_id": retweet_id,
        "user_id": user_id,
        "text": text,
        "likes": likes,
        "retweets": retweets,
        "safety_status": safety_status
    }
    response = requests.post(f'{base_url}/store_tweet', json=data)
    return response.json()

In [26]:
def simulate_user_submission():
    tweets = [
        # {"text": "Ths is a tweet for testing purposes.", "user": "example_user"},
        # {"text": "Anther test tweet for batch prediction.", "user": "example_user"},
        {"text": "nd yet another example of a tweet", "user": "example_user"}
    ]

    for tweet in tweets:
        prediction = post_tweet_for_prediction(tweet["text"], tweet["user"])
        print("Prediction Result:", prediction)
        
        if prediction["logreg_result"] != 1:
            store_response = store_posted_tweet(
                tweet_id=prediction["tweet_id"],
                retweet_id=None,
                user_id=tweet["user"],
                text=prediction["tweet"],
                likes=prediction["likes"],
                retweets=prediction["retweets"],
                safety_status=None  # Initial safety status is None
            )
            print("Store Tweet Response:", store_response)

In [27]:
simulate_user_submission()

Prediction Result: {'tweet_id': '327f4fca', 'tweet': 'nd yet another example of a tweet', 'user': 'example_user', 'likes': 0, 'retweets': 0, 'logreg_prob': 0.340711603105755, 'logreg_result': 0, 'cnn_prob': None, 'cnn_result': None}
Store Tweet Response: {'tweet_id': '327f4fca', 'text': 'nd yet another example of a tweet', 'retweet_id': None, 'retweets': 0, 'user_id': 'example_user', 'id': 4, 'likes': 0, 'safety_status': None}


In [117]:
time.sleep(40)  # Wait to ensure the scheduler runs at least once

## admin viewing tweets

In [28]:
# Function to fetch all tweets
def fetch_all_tweets():
    response = requests.get(f'{base_url}/fetch_tweets')
    return response.json()

In [29]:
# Fetch and display tweets after batch prediction
tweets_for_admin_view = fetch_all_tweets()

before scheduled prediction:

In [30]:
for i in range(len(tweets_for_admin_view)):
    print(tweets_for_admin_view[i])

{'tweet_id': 'fe4745d5', 'retweet_id': None, 'tweet': 'Ths is a tweet for testing purposes.', 'user': 'example_user', 'likes': 0, 'retweets': 0, 'logreg_prob': 0.0402386058008722, 'logreg_result': 0, 'cnn_prob': 0.0004499437927734107, 'cnn_result': 0, 'admin_result': None}
{'tweet_id': '7b6036f3', 'retweet_id': None, 'tweet': 'Anther test tweet for batch prediction.', 'user': 'example_user', 'likes': 0, 'retweets': 0, 'logreg_prob': 0.49591256501026953, 'logreg_result': 0, 'cnn_prob': 0.0665159821510315, 'cnn_result': 0, 'admin_result': None}
{'tweet_id': 'b29fcd13', 'retweet_id': None, 'tweet': 'nd yet another example of a tweet.', 'user': 'example_user', 'likes': 0, 'retweets': 0, 'logreg_prob': 0.340711603105755, 'logreg_result': 0, 'cnn_prob': 0.9057701826095581, 'cnn_result': 1, 'admin_result': 1}
{'tweet_id': '327f4fca', 'retweet_id': None, 'tweet': 'nd yet another example of a tweet', 'user': 'example_user', 'likes': 0, 'retweets': 0, 'logreg_prob': 0.340711603105755, 'logreg_re

after scheduled prediction:

In [31]:
# Fetch and display tweets after batch prediction
tweets_for_admin_view_after = fetch_all_tweets()

In [32]:
for i in range(len(tweets_for_admin_view_after)):
    print(tweets_for_admin_view_after[i])

{'tweet_id': 'fe4745d5', 'retweet_id': None, 'tweet': 'Ths is a tweet for testing purposes.', 'user': 'example_user', 'likes': 0, 'retweets': 0, 'logreg_prob': 0.0402386058008722, 'logreg_result': 0, 'cnn_prob': 0.0004499437927734107, 'cnn_result': 0, 'admin_result': None}
{'tweet_id': '7b6036f3', 'retweet_id': None, 'tweet': 'Anther test tweet for batch prediction.', 'user': 'example_user', 'likes': 0, 'retweets': 0, 'logreg_prob': 0.49591256501026953, 'logreg_result': 0, 'cnn_prob': 0.0665159821510315, 'cnn_result': 0, 'admin_result': None}
{'tweet_id': 'b29fcd13', 'retweet_id': None, 'tweet': 'nd yet another example of a tweet.', 'user': 'example_user', 'likes': 0, 'retweets': 0, 'logreg_prob': 0.340711603105755, 'logreg_result': 0, 'cnn_prob': 0.9057701826095581, 'cnn_result': 1, 'admin_result': 1}
{'tweet_id': '327f4fca', 'retweet_id': None, 'tweet': 'nd yet another example of a tweet', 'user': 'example_user', 'likes': 0, 'retweets': 0, 'logreg_prob': 0.340711603105755, 'logreg_re

In [33]:
# Print tweets where logreg_result does not match cnn_result
for tweet in tweets_for_admin_view_after:
    if tweet['logreg_result'] != tweet['cnn_result']:
        print(f"Discrepancy found for tweet_id {tweet['tweet_id']}:")
        print(f"  Logistic Regression Result: {tweet['logreg_result']}")
        print(f"  CNN Result: {tweet['cnn_result']}")
        print(f"  Tweet: {tweet['tweet']}\n")

Discrepancy found for tweet_id b29fcd13:
  Logistic Regression Result: 0
  CNN Result: 1
  Tweet: nd yet another example of a tweet.

Discrepancy found for tweet_id 327f4fca:
  Logistic Regression Result: 0
  CNN Result: 1
  Tweet: nd yet another example of a tweet



## user reporting tweets

In [34]:
import requests

# Example tweet ID to report and the safety status (e.g., 1 for hate speech)
report_data = {
    "tweet_id": "b29fcd13",
    'user_id': "example_user",
    "safety_status": 1  # Assume 1 indicates hate speech
}

# Function to report a tweet
def report_tweet(report_data):
    url = f"{base_url}/report_tweet"
    response = requests.post(url, json=report_data)
    
    if response.status_code == 200:
        print("Report submitted successfully.")
    else:
        print(f"Failed to submit report: {response.status_code}")
        print(f"Response: {response.json()}")




In [35]:
# Run the simulation
report_tweet(report_data)

Report submitted successfully.


## admin view reported tweets

In [36]:
# Function to fetch all reported tweets
def fetch_reported_tweets():
    url = f'{base_url}/reported_tweets'
    response = requests.get(url)
    return response.json()

# Fetch all reported tweets
reported_tweets = fetch_reported_tweets()

In [37]:
for i in range(len(reported_tweets)):
    print(reported_tweets[i])

{'tweet_id': 'b29fcd13', 'user_id': 'example_user', 'id': 1}
{'tweet_id': 'b29fcd13', 'user_id': 'example_user', 'id': 2}


## admin mark tweets unsafe

In [38]:
# Example data to mark a tweet as unsafe
update_data = {
    "tweet_id": "b29fcd13",
    "new_safety_status": 1,  # Assume 1 indicates hate speech
    "change_source": "admin"  # Indicates the change is made by an admin
}

In [39]:
# Function to update the safety status of a tweet
def update_safety_status(data):
    url = f"{base_url}/update_safety_status"
    response = requests.post(url, json=data)
    
    if response.status_code == 200:
        print("Safety status updated successfully.")
    else:
        print(f"Failed to update safety status: {response.status_code}")
        print(f"Response: {response.json()}")


In [40]:
# Simulate admin marking a tweet as unsafe
update_safety_status(update_data)

Safety status updated successfully.


## user display posted tweets

In [56]:
# Function to fetch all stored tweets for user view
def fetch_user_view_tweets():
    url = f"{base_url}/display_tweets"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch tweets: {response.status_code}")
        print(f"Response: {response.json()}")
        return None

In [57]:
# Fetch and display all user view tweets
user_view_tweets = fetch_user_view_tweets()

In [58]:
if user_view_tweets:
    for tweet in user_view_tweets:
        print(f"Tweet ID: {tweet['tweet_id']}")
        print(f"Retweet ID: {tweet.get('retweet_id', 'N/A')}")
        print(f"User ID: {tweet['user_id']}")
        print(f"Text: {tweet['text']}")
        print(f"Likes: {tweet['likes']}")
        print(f"Retweets: {tweet['retweets']}")
        print(f"Safety Status: {tweet.get('safety_status', 'N/A')}")
        print("-" * 40)
else:
    print("No tweets found.")

Tweet ID: fe4745d5
Retweet ID: None
User ID: example_user
Text: Ths is a tweet for testing purposes.
Likes: 0
Retweets: 0
Safety Status: None
----------------------------------------
Tweet ID: 7b6036f3
Retweet ID: None
User ID: example_user
Text: Anther test tweet for batch prediction.
Likes: 0
Retweets: 0
Safety Status: None
----------------------------------------
Tweet ID: b29fcd13
Retweet ID: None
User ID: example_user
Text: nd yet another example of a tweet.
Likes: 0
Retweets: 0
Safety Status: 1
----------------------------------------
Tweet ID: 327f4fca
Retweet ID: None
User ID: example_user
Text: nd yet another example of a tweet
Likes: 0
Retweets: 0
Safety Status: 1
----------------------------------------


In [44]:
for tweet in user_view_tweets:
    print(tweet)

{'tweet_id': 'fe4745d5', 'text': 'Ths is a tweet for testing purposes.', 'retweet_id': None, 'retweets': 0, 'user_id': 'example_user', 'id': 1, 'likes': 0, 'safety_status': None}
{'tweet_id': '7b6036f3', 'text': 'Anther test tweet for batch prediction.', 'retweet_id': None, 'retweets': 0, 'user_id': 'example_user', 'id': 2, 'likes': 0, 'safety_status': None}
{'tweet_id': 'b29fcd13', 'text': 'nd yet another example of a tweet.', 'retweet_id': None, 'retweets': 0, 'user_id': 'example_user', 'id': 3, 'likes': 0, 'safety_status': 1}
{'tweet_id': '327f4fca', 'text': 'nd yet another example of a tweet', 'retweet_id': None, 'retweets': 0, 'user_id': 'example_user', 'id': 4, 'likes': 0, 'safety_status': 1}


## admin fetch status changes 
made by admin or cnn model

In [45]:
# Function to fetch tweets with updated safety statuses for admin view
def fetch_safety_status_changes():
    url = f"{base_url}/safety_status_changes"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch safety status changes: {response.status_code}")
        print(f"Response: {response.json()}")
        return None

In [46]:
safety_status_changes = fetch_safety_status_changes()

In [47]:

if safety_status_changes:
    print("Fetched safety status changes:")
    for tweet in safety_status_changes:
        print(f"ID: {tweet['id']}, Tweet ID: {tweet['tweet_id']}, Safety Status: {tweet['new_safety_status']}, Change source: {tweet['change_source']}")

Fetched safety status changes:
ID: 1, Tweet ID: b29fcd13, Safety Status: 1, Change source: cnn
ID: 2, Tweet ID: b29fcd13, Safety Status: 1, Change source: admin
ID: 3, Tweet ID: 327f4fca, Safety Status: 1, Change source: cnn
ID: 4, Tweet ID: b29fcd13, Safety Status: 1, Change source: admin


## get tweet by id

In [59]:
# Function to fetch a tweet by ID
def fetch_tweet_by_id(tweet_id):
    url = f"{base_url}/fetch_tweet/{tweet_id}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

In [60]:
tweet = fetch_tweet_by_id('b29fcd13')
tweet

{'tweet_id': 'b29fcd13',
 'retweet_id': None,
 'user_id': 'example_user',
 'text': 'nd yet another example of a tweet.',
 'likes': 0,
 'retweets': 0,
 'safety_status': 1}